In [ ]:
#Code inspired by https://www.kaggle.com/code/helmehelmuto/secondary-structure-prediction-with-keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
from keras.utils import  to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
#Load the data
data = pd.read_csv('./Data/2018-06-06-ss.cleaned.csv')
data.len.hist(bins=100)
max_length = 128
print(data.shape)

#Remove sequences that are too long and sequences that contain non-standard amino acids
sequences, structures = data[['seq', 'sst3']][(data.len <= max_length) & (~data.has_nonstd_aa)].values.T 

#For saving memory
data = []

#Making the sequences into grams, because this makes it easier for the model to learn
def seq2ngrams(seqs, n=3):
    return np.array([[seq[i:i+n] for i in range(len(seq))] for seq in seqs])

sequences = seq2ngrams(sequences)


In [ ]:
#Preprocess the data

#Encode the sequences with padding
tokenizer_seq = Tokenizer()
tokenizer_seq.fit_on_texts(sequences)
sequences = tokenizer_seq.texts_to_sequences(sequences)
sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

print(sequences[1])

#sequence back to its string value
print(tokenizer_seq.sequences_to_texts(sequences[1:2])[0])

#Encode the structures, and categorize them
tokenizer_struc = Tokenizer(char_level=True)
tokenizer_struc.fit_on_texts(structures)
structures = tokenizer_struc.texts_to_sequences(structures)
structures = pad_sequences(structures, maxlen=max_length, padding='post')
structures = to_categorical(structures)

print(structures[1])

structures.shape, sequences.shape

In [6]:
word_amount = len(tokenizer_seq.word_index) + 1
tag_amount = len(tokenizer_struc.word_index) + 1

# define the model
model = Sequential()
model.add(Embedding(word_amount, 128, input_length=max_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(TimeDistributed(Dense(tag_amount, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(sequences, structures, test_size=0.2, random_state=42)

# train the model
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test), verbose=1)
model.save('modelv2.h5')

Epoch 1/10
555/555 [==============================] - 11s 18ms/step - loss: 1.1356 - accuracy: 0.4981 - val_loss: 1.1147 - val_accuracy: 0.5078
Epoch 2/10
555/555 [==============================] - 9s 17ms/step - loss: 0.9083 - accuracy: 0.5872 - val_loss: 0.8655 - val_accuracy: 0.6065
Epoch 3/10
555/555 [==============================] - 9s 17ms/step - loss: 0.7067 - accuracy: 0.6618 - val_loss: 0.7648 - val_accuracy: 0.6714
Epoch 4/10
555/555 [==============================] - 9s 16ms/step - loss: 0.5940 - accuracy: 0.7100 - val_loss: 0.7492 - val_accuracy: 0.6933
Epoch 5/10
555/555 [==============================] - 9s 17ms/step - loss: 0.5426 - accuracy: 0.7337 - val_loss: 0.7641 - val_accuracy: 0.7035
Epoch 6/10
555/555 [==============================] - 9s 16ms/step - loss: 0.5025 - accuracy: 0.7527 - val_loss: 0.7610 - val_accuracy: 0.7191
Epoch 7/10
555/555 [==============================] - 9s 16ms/step - loss: 0.4765 - accuracy: 0.7673 - val_loss: 0.7359 - val_accuracy: 0.732

In [ ]:
#Finding the amount of words and tags, so we can use it in the model, with the correct dimensions
"""

word_amount = len(tokenizer_seq.word_index) + 1
tag_amount = len(tokenizer_struc.word_index) + 1

# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim = word_amount, output_dim= 128, input_length=max_length))
model.add(Bidirectional(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model.add(Dense(tag_amount, activation='softmax'))
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sequences, structures, test_size=0.3, random_state=42)

# Train and save the model
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test), verbose=1)
model.save("protein_model2.h5")
"""

In [ ]:
model = model.load_model('protein_model2.h5')